# Strands Agents SDK と Claude 4 インタリーブドシンキングによるツールとしてのエージェント

このノートブックでは、Strands Agents SDK と Claude 4 の**インタリーブドシンキング**機能を使い、専門エージェントを連携させたインテリジェントなワークフローの構築方法を紹介します。

## インタリーブドシンキングとは

### インタリーブドシンキングとは？

インタリーブドシンキングは、Claude 4 モデルに搭載された新機能で、モデルが以下のことを可能にします：

1. **ツール呼び出しの合間に思考**：結果を処理・推論し、次のステップを決定
2. **複数ツールを推論しながら連鎖的に利用**：高度なマルチステップ意思決定が可能
3. **中間結果に応じて戦略を動的に適応**

### 仕組み

従来のエージェントのイベントループとインタリーブドシンキングのイベントループには多くの共通点があります：

```

クエリ → LLMが思考 → LLMがツール呼び出しを決定 → イベントループがツールを実行 → 出力がLLMに返る → [LLMがツール呼び出し不要または最終回答を出すまで繰り返し]

```

主な違いは、インタリーブドシンキングではイベントループがLLMの「思考」に基づいて動作する点です。従来はLLMの思考は隠されており、ツール呼び出しや最終回答が出るまで待つ必要がありました。

インタリーブドシンキングでは、LLMが「思考」している間にその内容がイベントループに「漏れ出し」、LLMが思考した時点でツールが即座に実行されます。つまり、LLMが最初の「決定」を下す時点で、すでに最終回答が得られていることが多いのです。

### 有効化方法

StrandsとBedrockでこの機能を有効化するには：
- `temperature=1` を設定（thinking有効時は必須）
- ベータヘッダー追加: `"anthropic_beta": ["interleaved-thinking-2025-05-14"]`
- 推論バジェット設定: `"reasoning_config": {"type": "enabled", "budget_tokens": 3000}`

## セットアップとインポート

In [ ]:
import os
from strands import Agent, tool
from strands.models import BedrockModel
from strands.models import bedrock

bedrock.DEFAULT_BEDROCK_MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

## 専門エージェント（ツール）の定義

まず、Strands の `@tool` デコレータを使って4つの専門エージェントを作成します：

- **リサーチャー**：事実情報の収集
- **データアナリスト**：情報の分析・処理
- **ファクトチェッカー**：情報の正確性検証
- **レポートライター**：最終レポートの作成

In [ ]:
# Specialist agents implemented as tools using Strands @tool decorator
@tool
def researcher(query: str) -> str:
    """
    Research specialist that gathers factual information.
    
    Args:
        query: Research question or topic to investigate
        
    Returns:
        Research findings and sources
    """
    # Create a focused research agent
    # Note: Each call creates a fresh agent instance (stateless)
    research_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
        system_prompt="You are a research specialist. Gather factual information and cite sources when possible. Keep responses under 200 words.",
        callback_handler=None  # No streaming for tool agents
    )
    
    # Execute the research task
    result = research_agent(f"Research: {query}")
    return str(result)

In [ ]:
@tool
def data_analyst(data: str) -> str:
    """
    Data analyst that processes and analyzes information.
    
    Args:
        data: Raw data or research findings to analyze
        
    Returns:
        Analysis with insights and patterns
    """
    # Analyst agent focuses on extracting insights
    analysis_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        system_prompt="You are a data analyst. Extract key insights, identify patterns, and provide analytical conclusions. Focus on actionable insights.",
        callback_handler=None
    )
    
    # Analyze the provided data
    result = analysis_agent(f"Analyze this data and provide insights: {data}")
    return str(result)

In [ ]:
@tool
def fact_checker(information: str) -> str:
    """
    Fact checker that verifies information accuracy.
    
    Args:
        information: Claims or data to verify
        
    Returns:
        Fact-check results with accuracy assessment
    """
    # Fact-checking agent for verification
    fact_check_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        system_prompt="You are a fact checker. Verify claims, assess credibility, and provide confidence levels. Identify any questionable statements.",
        callback_handler=None
    )
    
    # Verify the information
    result = fact_check_agent(f"Fact-check this information: {information}")
    return str(result)

In [ ]:
@tool
def report_writer(analysis: str) -> str:
    """
    Report writer that creates polished final documents.
    
    Args:
        analysis: Analyzed data and insights
        
    Returns:
        Formatted final report
    """
    # Writer agent for professional output
    writer_agent = Agent(
        system_prompt="You are a professional report writer. Create clear, well-structured reports with executive summaries and actionable recommendations.",
        callback_handler=None
    )
    
    # Create the report
    result = writer_agent(f"Create a professional report based on: {analysis}")
    return str(result)

## Claude 4 オーケストレーター（インタリーブドシンキング対応）

次に、インタリーブドシンキングを活用して専門エージェントを知的に連携させるClaude 4オーケストレーターを作成します。

### オーケストレーターの動作概要：

1. ユーザーから高レベルなタスクを受け取る
2. **必要な情報を思考**
3. researcherツールで初期データを収集
4. **リサーチ結果を思考し、必要な分析を考える**
5. data analystで分析
6. **正確性や検証の必要性を思考**
7. 必要に応じてfact checkerを呼び出す
8. **結果の提示方法を思考**
9. report writerで最終出力を作成
10. **全体のワークフローを振り返り、最終回答を返す**

In [ ]:
# Claude 4 Orchestrator with Interleaved Thinking using Strands
class StrandsInterlevedWorkflowOrchestrator:
    def __init__(self):
        # Define the orchestrator system prompt for intelligent workflow coordination
        self.system_prompt = """You are an intelligent workflow orchestrator with access to specialist agents.

        Your role is to intelligently coordinate a workflow using these specialist agents:
        - researcher: Gathers factual information on any topic
        - data_analyst: Analyzes data and extracts insights
        - fact_checker: Verifies accuracy of information  
        - report_writer: Creates polished final reports

        """
    
    def run_workflow(self, task: str, enable_interleaved_thinking: bool = True) -> str:
        """Execute an intelligent workflow for the given task.
        
        Args:
            task: The task to complete
            enable_interleaved_thinking: Whether to enable interleaved thinking (default: True)
        
        The orchestrator will:
        1. Understand the task requirements
        2. Think about the best approach
        3. Coordinate specialist agents
        4. Reflect on results between steps
        5. Produce a comprehensive output
        """
        thinking_mode = "WITH interleaved thinking" if enable_interleaved_thinking else "WITHOUT interleaved thinking"
        print(f"\nStarting intelligent workflow {thinking_mode} for: {task}")
        print("=" * 70)
        
        # Configure Claude 4 with or without interleaved thinking via Bedrock
        if enable_interleaved_thinking:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1,  # Required to be 1 when thinking is enabled
                additional_request_fields={
                    # Enable interleaved thinking beta feature
                    "anthropic_beta": ["interleaved-thinking-2025-05-14"],
                    # Configure reasoning parameters
                    "reasoning_config": {
                        "type": "enabled",  # Turn on thinking
                        "budget_tokens": 3000  # Thinking token budget
                    }
                }
            )
        else:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1
            )
        
        # Create the orchestrator agent with Claude 4 and specialist tools
        orchestrator = Agent(
            model=claude4_model,
            system_prompt=self.system_prompt,
            tools=[researcher, data_analyst, fact_checker, report_writer]
        )
        
        prompt = f"""Complete this task using intelligent workflow coordination: {task}

        Instructions:
        1. Think carefully about what information you need to accomplish this task
        2. Use the specialist agents strategically - each has unique strengths
        3. After each tool use, reflect on the results and adapt your approach
        4. Coordinate multiple agents as needed for comprehensive results
        5. Ensure accuracy by fact-checking when appropriate
        6. Provide a comprehensive final response that addresses all aspects
        
        Remember: Your thinking between tool calls helps you make better decisions.
        Use it to plan, evaluate results, and adjust your strategy.
        """
        
        try:
            result = orchestrator(prompt)
            return str(result)
        except Exception as e:
            return f"Workflow failed: {e}"

## デモ実行

オーケストレーターの動作を実際に見てみましょう！思考しながらツールを呼び出す様子に注目してください。

In [ ]:
# Create the orchestrator
print("Strands Agents SDK: Claude 4 Interleaved Thinking Workflow Demo")
print("=" * 70)

try:
    orchestrator = StrandsInterlevedWorkflowOrchestrator()
    print("✅ Orchestrator initialized successfully!")
except Exception as e:
    print(f"❌ Failed to initialize orchestrator: {e}")

In [ ]:
# Run the workflow with a test case
test_case = "Analyze the impact of remote work on productivity and provide strategic recommendations"

print(f"📋 Task: {test_case}\n")

try:
    result = orchestrator.run_workflow(test_case)
    
    print(f"\n📊 Workflow Result:")
    print("=" * 70)
    print(result)
except Exception as e:
    print(f"❌ Workflow execution failed: {e}")

## インタリーブドシンキングなしで試す

オーケストレーターをインタリーブドシンキング無効で実行し、出力の違いを観察してみましょう。

In [ ]:
# Now let's try the same task WITHOUT interleaved thinking
print("\n" + "="*70)
print("🔄 Now running the same task WITHOUT interleaved thinking")
print("="*70)

try:
    result_without_thinking = orchestrator.run_workflow(test_case, enable_interleaved_thinking=False)
    
    print(f"\n📊 Workflow Result (Without Interleaved Thinking):")
    print("=" * 70)
    print(result_without_thinking)
except Exception as e:
    print(f"❌ Workflow execution failed: {e}")